# The litdjango CLI


In order to adapt Django to a literate programming style using Jupyter notebooks, we need to
reimplement any builtin `django-admin` and `manage.py` cli commands which produce `.py` modules
into commands which produce `.ipynb` files.

In [ ]:
#| default_exp cli

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from collections import defaultdict
import importlib
import os
from pathlib import Path
import sys

from django.core.management import ManagementUtility, execute_from_command_line, get_commands
from django.core.management.base import BaseCommand
from django.core.management.color import color_style
from django.core.management.commands.startapp import Command as StartApp
from django.core.management.commands.startproject import Command as StartProject

from litdjango.utils import LITDJANGO_ROOT

## Subcommands

In [ ]:
#| export
class StartProject(StartProject):

    rewrite_template_suffixes = (('.py-tpl', '.py'), (('.ipynb-tpl', '.ipynb')))

    def handle(self, **options):
        options["template"] = str(LITDJANGO_ROOT / "templates" / "project_template")
        options["extensions"] = ['py', 'txt', 'ipynb']
        super().handle(**options)

In [ ]:
StartProject.__mro__

(__main__.StartProject,
 django.core.management.commands.startproject.Command,
 django.core.management.templates.TemplateCommand,
 django.core.management.base.BaseCommand,
 object)

In [ ]:
#| export
class Export(BaseCommand):
    help = "Export .ipynb notebooks to .py modules"
    
    def handle(self, *args, **options):
        self.stdout.write("Exporting...")

In [ ]:
#| export
class StartApp(StartApp):
    pass

## Register subcommands with the cli

In [ ]:
#| exporti
lit_commands = {
    "startapp": StartApp(), # overrides django default
    "export": Export(), # new command
    "startproject": StartProject() #overrides django default
}

In [ ]:
#| export
class LitManagementUtility(ManagementUtility):
    """The litdjango cli is an instance of this class
    
    We use the lit_commands dict as a container for new litdjango commands, or 
    overridden django commands, and fallback to the default django command for anything
    else.
    """
    def __init__(self, argv=None):
        super().__init__(argv)
        
    def main_help_text(self, commands_only=False):
        """Return the script's main help text, as a string."""
        if commands_only:
            usage = sorted(set(get_commands()) | set(lit_commands))  # overwrite django
        else:
            usage = [
                "",
                "Type '%s help <subcommand>' for help on a specific subcommand." % self.prog_name,
                "",
                "Available subcommands:",           
                ]
            commands_dict = defaultdict(lambda: []) 
            commands_dict["litdjango"] = sorted(lit_commands.keys())
            for name, app in get_commands().items():
                if app == "django.core":
                    if name in lit_commands.keys():
                        continue
                    else:
                        app = "django"
                else:
                    app = app.rpartition(".")[-1]
                commands_dict[app].append(name)
            style = color_style()
            for app in sorted(commands_dict):
                usage.append("")
                usage.append(style.NOTICE("[%s]" % app))
                for name in sorted(commands_dict[app]):
                    usage.append("    %s" % name)
            # Output an extra note if settings are not properly configured
            if self.settings_exception is not None:
                usage.append(
                    style.NOTICE(
                        "Note that only Django core commands are listed "
                        "as settings are not properly configured (error: %s)."
                        % self.settings_exception
                    )
                )

        return "\n".join(usage)


    def fetch_command(self, subcommand):
        """Use lit_commands version if it exists, otherwise fallback to django commands"""
        try: # find a lit_command version of the command
            return lit_commands[subcommand]
        except KeyError:
            # Fall back to default django  if we have not defined a custom command
            return super().fetch_command(subcommand)

In [ ]:
sorted(lit_commands.keys())

['export', 'startapp', 'startproject']

In [ ]:
from django.core.management import get_commands

sorted(set(get_commands()) | set(lit_commands))
commands_dict = defaultdict(lambda: [])
commands_dict["litdjango"] = sorted(lit_commands.keys())
for name, app in get_commands().items():
    if app == "django.core":
        if name in lit_commands.keys():
            continue
        app = "django"
    else:
        app = app.rpartition(".")[-1]
    print(app, name)
    commands_dict[app].append(name)



django check
django compilemessages
django createcachetable
django dbshell
django diffsettings
django dumpdata
django flush
django inspectdb
django loaddata
django makemessages
django makemigrations
django migrate
django optimizemigration
django runserver
django sendtestemail
django shell
django showmigrations
django sqlflush
django sqlmigrate
django sqlsequencereset
django squashmigrations
django test
django testserver


defaultdict(<function __main__.<lambda>()>,
            {'litdjango': ['export', 'startapp', 'startproject'],
             'django': ['check',
              'compilemessages',
              'createcachetable',
              'dbshell',
              'diffsettings',
              'dumpdata',
              'flush',
              'inspectdb',
              'loaddata',
              'makemessages',
              'makemigrations',
              'migrate',
              'optimizemigration',
              'runserver',
              'sendtestemail',
              'shell',
              'showmigrations',
              'sqlflush',
              'sqlmigrate',
              'sqlsequencereset',
              'squashmigrations',
              'test',
              'testserver']})

In [ ]:
#| export
def cli():
    """This is set as entrypoint of the litdjango command in the package's setup.py"""
    utility = LitManagementUtility(sys.argv)
    utility.execute()

No Django settings specified.
Unknown command: '--ip=127.0.0.1'
Type 'ipykernel_launcher.py help' for usage.


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
!litdjango startproject --help

usage: litdjango startproject [-h] [--template TEMPLATE]
                              [--extension EXTENSIONS] [--name FILES]
                              [--exclude [EXCLUDE]] [--version] [-v {0,1,2,3}]
                              [--settings SETTINGS] [--pythonpath PYTHONPATH]
                              [--traceback] [--no-color] [--force-color]
                              name [directory]

Help from lit django. Creates a Django project directory structure for the
given project name in the current directory or optionally in the given
directory.

positional arguments:
  name                  Name of the application or project.
  directory             Optional destination directory

options:
  -h, --help            show this help message and exit
  --template TEMPLATE   The path or URL to load the template from.
  --extension EXTENSIONS, -e EXTENSIONS
                        The file extension(s) to render (default: "py").
                        Separate multiple extensions w

<!--  -->